# 2. 온라인 선호지수 개발

## 2.2 협업 필터링을 통한 선호도 점수 행렬 도출

In [94]:
import numpy as np
import pandas as pd
import sklearn
from pandas import Series, DataFrame
import seaborn as sns
import matplotlib.pyplot as plt
from pathlib import Path
import os

In [3]:
#데이터가져오기(기본 데이터)
Product = pd.read_csv("C:/Users/lsh93/Desktop/data/Product.csv")
Search1 = pd.read_csv("C:/Users/lsh93/Desktop/data/Search1.csv")
Search2 = pd.read_csv("C:/Users/lsh93/Desktop/data/Search2.csv")
Custom = pd.read_csv("C:/Users/lsh93/Desktop/data/Custom.csv")
Session = pd.read_csv("C:/Users/lsh93/Desktop/data/Session.csv")
Master = pd.read_csv("C:/Users/lsh93/Desktop/data/Master.csv")

C:\Users\lsh93\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3020: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
#데이터가져오기(created from 'Developing Online Preference 1')
Client_Behavior = pd.read_csv("C:/Users/lsh93/Desktop/data/UBCF/Client_Behavior.csv")
Product_Used = pd.read_csv("C:/Users/lsh93/Desktop/data/UBCF/Product_Used.csv")
Client_Used =  pd.read_csv("C:/Users/lsh93/Desktop/data/UBCF/Client_Used.csv")
Custom_Used = pd.read_csv("C:/Users/lsh93/Desktop/data/UBCF/Custom_Used.csv")
Search1_Used = pd.read_csv("C:/Users/lsh93/Desktop/data/UBCF/Search1_Used.csv")
Session_Used = pd.read_csv("C:/Users/lsh93/Desktop/data/UBCF/Session_Used.csv")

C:\Users\lsh93\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3020: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [31]:
#앞에 unnamed가 붙여진 건 삭제해줘야 할듯,, 자동으로 index column붙은 듯 
#데이터 보기
Client_Behavior[0:1]

,Unnamed: 0,CLNT_ID,SESS_ID,HITS_SEQ,PD_C,PD_ADD_NM,PD_BRA_NM,PD_BUY_AM,PD_BUY_CT,SESS_SEQ,...,TOT_SESS_HR_V,DVC_CTG_NM,ZON_NM,CITY_NM,PD_NM,CLAC1_NM,CLAC2_NM,CLAC3_NM,KWD_NM,SEARCH_CNT
0,0,4148106,6662803,94,570603,색상:WHT0_WHT0-(WHITE)|사이즈:115 / 1개,데상트,"39,000",1,55,...,"1,371",mobile,Seoul,Seoul,[데상트] 스포츠베이직 빅로고 티셔츠 - DMGR_DMELANGE / 115,스포츠패션,남성일반스포츠의류,남성스포츠티셔츠,라코스떼 남성티셔츠,2


In [32]:
Product_Used[0:1]

,CLNT_ID,SESS_ID,HITS_SEQ,PD_C,PD_ADD_NM,PD_BRA_NM,PD_BUY_AM,PD_BUY_CT
0,4140076,10189797,13,570603,색상:BLK0_(BLK0)BLACK|사이즈:120 / 2개,데상트,"39,000",2


In [33]:
Client_Used[0:1]

,CLNT_ID
87,2268789


1) 구매 count를 이용한 온라인 선호지수 생성

In [35]:
#데이터 불러오기
Purchase_Count = pd.read_csv("C:/Users/lsh93/Desktop/data/UBCF/Purchase_Count.csv")

In [36]:
Purchase_Count.head()

,Unnamed: 0,스포츠패션,남성의류,유아동의류,청소/세탁/욕실용품,완구,패션잡화,시즌스포츠,여성의류,인테리어/조명,...,음료,구기/필드스포츠,냉장/세탁가전,냉동식품,냉장식품,원예/애완,상품권,자동차용품,축산물,영상/음향가전
0,4140076,23.0,52.0,17.0,5.0,5.0,24.0,8.0,8.0,7.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,4148106,3.0,3.0,0.0,0.0,0.0,9.0,7.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,5298612,1.0,4.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,5553084,7.0,13.0,21.0,0.0,3.0,10.0,11.0,19.0,0.0,...,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5639922,11.0,16.0,9.0,0.0,0.0,17.0,4.0,55.0,0.0,...,0.0,0.0,0.0,0.0,0.0,16.0,0.0,0.0,0.0,0.0


In [39]:
Purchase_Count = Purchase_Count.rename(columns = {'Unnamed: 0':'CLNT_ID'})

In [40]:
Purchase_Count.head()

,CLNT_ID,스포츠패션,남성의류,유아동의류,청소/세탁/욕실용품,완구,패션잡화,시즌스포츠,여성의류,인테리어/조명,...,음료,구기/필드스포츠,냉장/세탁가전,냉동식품,냉장식품,원예/애완,상품권,자동차용품,축산물,영상/음향가전
0,4140076,23.0,52.0,17.0,5.0,5.0,24.0,8.0,8.0,7.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,4148106,3.0,3.0,0.0,0.0,0.0,9.0,7.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,5298612,1.0,4.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,5553084,7.0,13.0,21.0,0.0,3.0,10.0,11.0,19.0,0.0,...,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5639922,11.0,16.0,9.0,0.0,0.0,17.0,4.0,55.0,0.0,...,0.0,0.0,0.0,0.0,0.0,16.0,0.0,0.0,0.0,0.0


In [87]:
row_names = Purchase_Count['CLNT_ID'].as_matrix()
row_names

C:\Users\lsh93\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.


array([4140076, 4148106, 5298612, ..., 4362431, 6510160, 1835621],
      dtype=int64)

In [72]:
col_names = Purchase_Count.columns.values
col_names = col_names[1:]

In [74]:
temp_matrix = Purchase_Count.as_matrix()

C:\Users\lsh93\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.


In [81]:
temp_matrix = temp_matrix[:,1:]

In [86]:
temp_matrix.shape

(375839, 37)

In [88]:
temp = pd.DataFrame(temp_matrix, columns = col_names, index = row_names)

In [89]:
temp

,스포츠패션,남성의류,유아동의류,청소/세탁/욕실용품,완구,패션잡화,시즌스포츠,여성의류,인테리어/조명,속옷/양말/홈웨어,...,음료,구기/필드스포츠,냉장/세탁가전,냉동식품,냉장식품,원예/애완,상품권,자동차용품,축산물,영상/음향가전
4140076,23.0,52.0,17.0,5.0,5.0,24.0,8.0,8.0,7.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4148106,3.0,3.0,0.0,0.0,0.0,9.0,7.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5298612,1.0,4.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5553084,7.0,13.0,21.0,0.0,3.0,10.0,11.0,19.0,0.0,6.0,...,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5639922,11.0,16.0,9.0,0.0,0.0,17.0,4.0,55.0,0.0,12.0,...,0.0,0.0,0.0,0.0,0.0,16.0,0.0,0.0,0.0,0.0
5650295,2.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5851368,8.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4128075,10.0,0.0,0.0,0.0,0.0,1.0,8.0,8.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4207528,3.0,2.0,1.0,2.0,0.0,7.0,0.0,5.0,2.0,9.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4219875,5.0,0.0,0.0,6.0,0.0,2.0,0.0,2.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [90]:
Purchase_Count = temp
Purchase_Count.head()

,스포츠패션,남성의류,유아동의류,청소/세탁/욕실용품,완구,패션잡화,시즌스포츠,여성의류,인테리어/조명,속옷/양말/홈웨어,...,음료,구기/필드스포츠,냉장/세탁가전,냉동식품,냉장식품,원예/애완,상품권,자동차용품,축산물,영상/음향가전
4140076,23.0,52.0,17.0,5.0,5.0,24.0,8.0,8.0,7.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4148106,3.0,3.0,0.0,0.0,0.0,9.0,7.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5298612,1.0,4.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5553084,7.0,13.0,21.0,0.0,3.0,10.0,11.0,19.0,0.0,6.0,...,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5639922,11.0,16.0,9.0,0.0,0.0,17.0,4.0,55.0,0.0,12.0,...,0.0,0.0,0.0,0.0,0.0,16.0,0.0,0.0,0.0,0.0


In [119]:
#코딩 효율성 
Purchase_Count[1:]

SyntaxError: invalid syntax (<ipython-input-119-49ec3cf85a09>, line 2)

In [49]:
#Array로 변환
PC = PC.as_matrix()

C:\Users\lsh93\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.


In [122]:
PC = PC[1:,1:]

In [123]:
PC

array([[ 3.,  3.,  0., ...,  0.,  0.,  0.],
       [ 1.,  4.,  2., ...,  0.,  0.,  0.],
       [ 7., 13., 21., ...,  0.,  0.,  0.],
       ...,
       [ 2.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  1.,  0., ...,  0.,  0.,  0.],
       [ 2.,  0.,  0., ...,  0.,  0.,  0.]])

In [124]:
sparsity = float(len(PC.nonzero()[0]))
sparsity /= (PC.shape[0] * PC.shape[1])
sparsity *= 100
print('Sparsity: {:4.2f}%',format(sparsity))

Sparsity: {:4.2f}% 7.94012313816059


In [97]:
from sklearn.model_selection import train_test_split

In [98]:
from sklearn.cross_validation import train_test_split

ModuleNotFoundError: No module named 'sklearn.cross_validation'

In [125]:
PC_train, PC_test = train_test_split(PC, test_size = 0.33, random_state = 42)

In [126]:
print(PC_train.shape)
print(PC_test.shape)

(251811, 37)
(124027, 37)


In [133]:
from sklearn.metrics import pairwise_distances
#from sklearn.metrics.pairwise import cosine_similarity
from scipy.spatial.distance import cosine

In [152]:
temp = PC[0:10000,0:]

In [148]:
temp

array([[ 3.,  3.,  0., ...,  0.,  0.,  0.],
       [ 1.,  4.,  2., ...,  0.,  0.,  0.],
       [ 7., 13., 21., ...,  0.,  0.,  0.],
       ...,
       [ 1.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.]])

In [154]:
pairwise_distances(temp, metric="cosine")

array([[0.        , 0.730939  , 0.49477153, ..., 0.36647703, 0.57268644,
        0.75607873],
       [0.730939  , 0.        , 0.40323335, ..., 0.21854097, 0.47797321,
        0.55404825],
       [0.49477153, 0.40323335, 0.        , ..., 0.48777931, 0.19344928,
        0.29127282],
       ...,
       [0.36647703, 0.21854097, 0.48777931, ..., 0.        , 0.70990373,
        0.81693219],
       [0.57268644, 0.47797321, 0.19344928, ..., 0.70990373, 0.        ,
        0.1480706 ],
       [0.75607873, 0.55404825, 0.29127282, ..., 0.81693219, 0.1480706 ,
        0.        ]])

In [108]:
dist_out

NameError: name 'dist_out' is not defined